
Basic usage of the MPDS API
==========

- <font color=blue>Complexity level: beginner</font>

Let's play a bit with the MPDS API?


In [ ]:
!pip install mpds_client>=0.0.17

In [ ]:
from mpds_client import MPDSDataRetrieval, MPDSDataTypes, APIError

In [ ]:
[x for x in dir(MPDSDataTypes) if not x.startswith('__')]

In future we'll also add _ab initio_ data, but peer-reviewed data are (and will remain) default.

In [ ]:
example_props = [ # NB these props support machine-learning data type
'isothermal bulk modulus',
'enthalpy of formation',
'heat capacity at constant pressure',
'Seebeck coefficient',
'values of electronic band gap', # NB both direct + indirect gaps
'temperature for congruent melting',
'Debye temperature',
'linear thermal expansion coefficient'
]

Default fields make sense, but we'll additionally customize them below.

In [ ]:
desired_fields = {
    'P':[ # *P*hysical property entries
        'sample.material.entry',
        'sample.material.phase',
        'sample.material.chemical_elements',
        'sample.material.chemical_formula'
    ],
    'S':[ # Crystalline *S*tructure entries
        'entry'
        'phase',
        'chemical_elements',
        'chemical_formula'
    ],
    'C':[ # Phase diagrams, i.e. *C*onstitution entries
        'entry',
        lambda: 'MANY-PHASE', # constants are given like this (on purpose)
        'chemical_elements',
        lambda: 'MANY-FORMULAE'
    ]
    # NB. P-S-C are interconnected by means of the distinct phases
}


Copy and paste your [MPDS API key](https://mpds.io/open-data-api) in the next cell, then execute. Note, if the key isn't valid, the API returns an HTTP error `403`.

Please, make sure not to expose your MPDS key publicly.


In [ ]:
client = MPDSDataRetrieval(dtype=MPDSDataTypes.MACHINE_LEARNING, api_key='YOUR_MPDS_API_KEY_GOES_HERE')

In [ ]:
for prop in example_props:

    print("*" * 100)
    print("Considering %s" % prop)

    try:
        for card in client.get_data({
            "props": prop,
            "classes": "transitional, superconductor",
            # any atom of that group (transitional metal) must be present
            # + physics term (superconductor) assigned in the original publication
            "aetypes": "all 7-vertex",
            # e.g. hexagonal pyramid, pentagonal bipyramid etc.
            "aeatoms": "X-S",
            # any atom in the center, sulphur in the vertices (ligands)
            "years": "2010-2019"
            # only recent results (void for MACHINE_LEARNING, as all are 2018)
        }, fields=desired_fields):

            print("%s %s %s" % (card[0], "-".join(card[2]), card[3]))

    except APIError as ex:

        if ex.code == 1:
            print("No matches.")

        else:
            print("Error %s: %s" % (ex.code, ex.msg))

In [ ]:
client.dtype = MPDSDataTypes.PEER_REVIEWED

print(client.get_data({"elements": "O", "classes": "binary", "sgs": "I4/mmm"}))

In [ ]:
import random
prop = random.choice(example_props)

print(client.get_data({"props": prop, "elements": "O", "classes": "binary, lanthanoid, non-disordered"}))


Were you able to follow everything? Please, try to explain, what happens under the hood approximately, when we call `client.get_data` or `client.get_dataframe`.
